User Manual for Tree

# preparing processes

## In this part, We import packages that we need to use.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from mlxtend.evaluate import bias_variance_decomp

## Then, we transfer catagorical data to numerical representations to run regression

* For catagorical data ShelveLoc, we tranfer Good to 2, Meduim to 1 and bad to 0. 

* For catagorical data Urban and US, we transfer Yes to 1, No to 0.

In [ ]:
# import data
df = pd.DataFrame(pd.read_csv("Carseats.csv"))
# transform classes into numerical value
df['ShelveLoc'].replace(['Good', 'Medium', 'Bad'],[2,1,0], inplace=True)
df['Urban'].replace(['Yes', 'No'], [1,0], inplace = True)
df['US'].replace(['Yes', 'No'], [1,0], inplace = True)

## Then we split training dataset and testing dataset.

### Approach 1: utilize pandas

We randomly pick 80% data as training data, other 20% as test data. 

In [ ]:
# reorder indexes
df = df.sample(frac=1.0)
df = df.reset_index()
# drop column of index
df = df.drop("index", axis = 1)

# divide training dataset and testing dataset
# train
train = df.sample(frac=0.8)
train = train.reset_index(drop=True)

train_x = train.drop(["Sales"], axis=1)
train_y = train["Sales"]
#test
test = df[~df.index.isin(train.index)]
test = test.reset_index(drop=True)

test_x = test.drop(["Sales"], axis=1)
test_y = test["Sales"]

### approach 2: use train_test_split method in package sklearn.model_selection

the effect is equivalent

In [ ]:
from sklearn.model_selection import train_test_split # 划分数据集的模块
train_x, test_x, train_y, test_y = train_test_split(df.drop(["Sales"], axis=1),df["Sales"],test_size=0.2)

# (a) illustrate data statistics

## We first describe the variables

In [ ]:
df.describe()

In [ ]:
print(train_x.shape, test_x.shape)

## Then we plot histograms

### Sales

else are performed in the same way

In [ ]:
fig, ax = plt.subplots()
ax.hist(df.iloc[:,0],edgecolor="white")
ax.set_title('Histogram For Sales')
ax.set_xlabel('Sales')

# grow and then plot the decision tree

## parameter explanation

As we use SSE (sum of squared error) as loss, we adopt the inherited "squared error" approach since it uses MSE, which is the same as using SSE (MSE = SSE/n), as default

## formula under the model

### for each leaf

e := MSE = $\frac{sum_{i=1}^{n} (y_i - \bar{y})^2} {n}$

### Total Mse is computed as:

$ S = \sum_{m \in leaves} e_m $

### After training model, we have Training MSE. Then we can also compute Test MSE with the trained model.

## coding

### preparing functions

In [ ]:
# compute training and testing MSE
def compute_mse(clf,x,y):
    predicted = clf.predict(x)
    mse = 0
    for i in range(len(x)):
        error_sqr = (y[i] - predicted[i])**2
        mse += error_sqr
    mse /= len(x)
    return mse

# train regression tree with max_depth, min_samples_leaf and then return the train_score, train_mse, test_score and test_mse
def compute_accuracy(max_depth, min_samples_leaf):
    global train_x, train_y, test_x, test_y
    clf = tree.DecisionTreeRegressor(max_depth=max_depth,min_samples_leaf = min_samples_leaf)
    clf = clf.fit(train_x, train_y)

    # compute training accuracy
    train_score = clf.score(train_x, train_y) # return R^2 of fitting
    test_score = clf.score(test_x, test_y) # return R^2 of fitting

    train_mse = compute_mse(clf,train_x, train_y)
    test_mse = compute_mse(clf,test_x,test_y)

    return clf, train_score, train_mse, test_score, test_mse

### define function to plot tree given max depth and min sample leaf

### approach 1 to plot tree: using plot_tree method in tree

In [ ]:
result = list()
def output_result(max_depth, min_samples_leaf):
    clf, train_score, train_mse, test_score, test_mse = compute_accuracy(max_depth=max_depth,min_samples_leaf=min_samples_leaf)
    
    # print the train and testing accuracy
    name = f"depth - {max_depth} minimum samples leaf - {min_samples_leaf}" + ".jpg"
    print("------------------------------------------------------------------------------------------------------------------")
    print(f"-----------------Regression Tree with maximum depth = {max_depth} and minimun number of sample in a leaf = {min_samples_leaf}----------------")
    print("Train accuracy: " , train_score , "\nTraining MSE: " , train_mse , "\nTesting accuracy: " , test_score , "\nTest MSE: " , test_mse)
    # print("------------------------------------------------------------------------------------------------------------------\n")
    # plot the tree
    plt.figure(figsize=(25,25))

    tree.plot_tree(clf,fontsize=9)
    
    plt.savefig(name)
    result.append([max_depth, min_samples_leaf, train_score, train_mse, test_score, test_mse])
    print("------------------------------------------------------------------------------------------------------------------\n")


### approach 2 to plot tree: using package graphviz.Source(dot_data)

the example is from a classification tree

In [ ]:
import graphviz
dot_data = tree.export_graphviz(clf #训练好的模型
                                ,out_file = None
                                ,feature_names= feature_name
                                ,class_names=["琴酒","雪莉","贝尔摩德"]
                                ,filled=True #进行颜色填充
                                ,rounded=True #树节点的形状控制
)
graph = graphviz.Source(dot_data)
graph

## draw trees for models:

* model1: max depth = 3 and min sample leaf = 5

* model2: max depth = 3 and min sample leaf = 10

* model3: max depth = 3 and min sample leaf = 15

* model4: max depth = 5 and min sample leaf = 10

* model5: max depth = 7 and min sample leaf = 10


In [ ]:
# output_result(3,5)
# output_result(3,10)
# output_result(3,15)
# output_result(5,10)
# output_result(7,10)

## To sum up, we have training result and testing result as follows:

|max_depth | min_samples_leaf | train_score | train_mse | test_score | test_mse |
| --- | --- | --- | --- | --- | --- |
|3 | 5 | 0.5513142619684648 | 3.469907690714441 | 0.3429246728518772 | 3.89318955376371|
|3 | 10 | 0.5456586169740507 | 3.5136455776108786 | 0.3236975350499671 | 4.007110878985372|
|3 | 15 | 0.5341587592558981 | 3.602579638482532 | 0.30315024242726696 | 4.128854158167713|
|5 | 10 | 0.6776681795467121 | 2.4927506447157017 | 0.5338846321452615 | 2.7617464939019363|
|7 | 10 | 0.7260161022281123 | 2.118852357958776 | 0.6196534289927718 | 2.253563991638637|

## if we want to do with pruning manually and get correlated information

In [ ]:
#特征重要性
clf.feature_importances_# 查看每一个特征对分类的贡献率
[*zip(feature_name,clf.feature_importances_)]

In [ ]:
# 在这里，我们发现每一次运行程序时，返回的准确率都不相同，这是因为sklearn每次都在全部的特征中选取若干个特征建立一棵树
# 最后选择准确率最高的决策树返回，如果我们添加上random_state参数，那么sklearn每一次建立决策树使用的特征都相同，返回的
# 预测分数也会一样clf = tree.DecisionTreeClassifier(criterion="entropy",random_state=30)
clf = clf.fit(Xtrain, Ytrain)
score = clf.score(Xtest, Ytest) #返回预测的准确度
score

### knowing the basic information, we need to prune tree to reduce overfitting

#### check if overfitting for training data

score_train = clf.score(Xtrain, Ytrain)
# 对于训练数据集的预测分数为100%，也就是过拟合了，需要我们做剪枝处理
score_train 

# if overfitting, we pose limit to min_samples_leaf and min_samples_split

clf1 = tree.DecisionTreeClassifier(criterion="entropy"
                                  ,random_state=30
                                  ,splitter="random"
                                  ,max_depth=3
                                  ,min_samples_leaf=10 #一个节点分支后，每一个子节点至少包含10个样本
                                  ,min_samples_split=10 #一个节点至少包含10个样本才会分支
)
clf1=clf1.fit(Xtrain,Ytrain)#拟合模型
dot_data=tree.export_graphviz(clf,feature_names=feature_name,class_names=["琴酒","雪莉","贝尔摩德"],filled=True,rounded=True)
graph=graphviz.Source(dot_data)
graph

#### 精修树的特征

max_features & min_impurity_decrease

一般max_depth使用，用作树的”精修“。max_features限制分枝时考虑的特征个数，超过限制个数的特征都会被舍弃。和max_depth异曲同工，max_features是用来限制高维度数据的过拟合的剪枝参数，但其方法比较暴力，是直接限制可以使用的特征数量而强行使决策树停下的参数，在不知道决策树中的各个特征的重要性的情况下，强行设定这个参数可能会导致模型学习不足。如果希望通过降维的方式防止过拟合，建议使用PCA，ICA或者特征选择模块中的降维算法。

min_impurity_decrease限制信息增益的大小，信息增益小于设定数值的分枝不会发生。这是在0.19版本中更新的功能，在0.19版本之前时使用min_impurity_split。

#### get the best hyper-parameter: draw learning plot (error v.s. hyperparameter)

In [ ]:
import matplotlib.pyplot as plt
test = []
for i in range(10):
    clf = tree.DecisionTreeClassifier(max_depth=i+1
                                      ,criterion="entropy"
                                      ,random_state=30
                                      ,splitter="random"
    )
    clf = clf.fit(Xtrain, Ytrain)
    score = clf.score(Xtest, Ytest)
    test.append(score)
plt.plot(range(1,11),test,color="red",label="max_depth")
plt.legend()
plt.show()

# bagging for trees

## Illustration

Here we try growing bagging tree for different depth and different number of trees.

## Growing steps

### Step 1:

Sample records with replacement, i.e. bootstrap training data to obtain several diverse training datasets

### Step2:

Fit the full model to each resampled training dataset.

### Step3:

Aggregate the predictions of all single trees. The average value is put as the prediction

## Coding

### We first define the function to output result

In [ ]:
result_bagging = list()
def output_bagging(number_of_tree, max_depth):
    global train_x, train_y, test_x, test_y
    base_clf = tree.DecisionTreeRegressor(max_depth=max_depth,min_samples_leaf = 5)
    clf = BaggingRegressor(base_estimator=base_clf,oob_score=True,n_estimators=number_of_tree,random_state=42)
    clf = clf.fit(train_x,train_y)

    train_mse = compute_mse(clf,train_x,train_y)
    test_mse = compute_mse(clf,test_x,test_y)
    accuracy = clf.oob_score_

    print("------------------------------------------------------------------------------------------------------------------")
    print(f"-----------------Regression Tree with maximum depth = {max_depth} and number of bagging trees = {number_of_tree}----------------")
    print("Training MSE: " , train_mse , "\nTest MSE: " , test_mse, "\nOOB Score: ", accuracy)
    print("------------------------------------------------------------------------------------------------------------------\n")

    result_bagging.append([max_depth, number_of_tree, train_mse,test_mse,accuracy])

## try different depth and different number of tree

* maximun depth = 10, number of tree = 10

* maximun depth = 10, number of tree = 20

* maximun depth = 10, number of tree = 50

* maximun depth = 10, number of tree = 100

* maximun depth = 10, number of tree = 200

* maximun depth = 3, number of tree = 50

* maximun depth = 5, number of tree = 50

* maximun depth = 15, number of tree = 50

In [ ]:
try_num_tree = [10,20,50,100,200]
for tree_num in try_num_tree:
    output_bagging(number_of_tree = tree_num, max_depth = 10)

output_bagging(number_of_tree=50,max_depth=3)

output_bagging(number_of_tree=50,max_depth=5)

output_bagging(number_of_tree=50,max_depth=15)

## To sum up， we summarize the output as table:

|max_depth | number_of_tree | train_mse | test_mse | OOB Score|
| --- | --- | --- | --- | --- |
|10 | 10 | 1.4282862942484493 | 1.4792816075410418 | 0.5740700266134335|
|10 | 20 | 1.2727152828703814 | 1.3233867244240793 | 0.6111830279416357|
|10 | 50 | 1.2498169388765552 | 1.4035065708634766 | 0.6341632588258435|
|10 | 100 | 1.2242041274803506 | 1.330610859881188 | 0.645983255682975|
|10 | 200 | 1.216718600689651 | 1.3483932890600459 | 0.6468993072337046|
|3 | 50 | 2.966433598730782 | 3.213220935261924 | 0.5076018639027922|
|5 | 50 | 1.6773229717753961 | 1.841179913558071 | 0.6063411599122327|
|15 | 50 | 1.2492917987531191 | 1.4027737677334209 | 0.6343230238269876|


# (d) Random forest

## Description:

### Basic idea: 

constructing several diverse decision trees and then combine their predicting results. By this model, we introduce randomness into data-level.

### aim: 

to reduce correlation among the trees produced by bagging, we introduce split-attribute randomization into model: random forest
	
## building algorithm:

each time a split is performed (search for the split attribute is limited to a random subset of m of the N attributes) 

That is, the number of feature a sub-tree use, the number of data samples a tree uses is limited. This reduces correlation among trees.

## formula

###	for regression tree: $m = \frac{N}{3}$
### for classification tree: $m=\sqrt N$

## Coding

### first define function to count error

In [ ]:
result_rf = list()
def output_rf(number_of_tree, m):
    global train_x, train_y, test_x, test_y
    clf = RandomForestRegressor(max_features=m, n_estimators=number_of_tree, oob_score=True, random_state=123)
    clf = clf.fit(train_x,train_y)

    train_mse = compute_mse(clf,train_x,train_y)
    test_mse = compute_mse(clf,test_x,test_y)


    print("------------------------------------------------------------------------------------------------------------------")
    print(f"-----------------maximum number of features = {m} and number of growing tree = {number_of_tree}----------------")
    print("Training MSE: " , train_mse , "\nTest MSE: " , test_mse)
    try:
        accuracy = clf.oob_score_
        print("\nOOB Score: ", accuracy)
    except:
        pass
    print("------------------------------------------------------------------------------------------------------------------\n")

    try:
        result_rf.append([number_of_tree, m, train_mse,test_mse,accuracy])
    except:
        result_rf.append([number_of_tree, m, train_mse,test_mse])
        

## we try different number of trees and values of m

* number of tree = 10,29,...,100, while m = 10

* number of tree = 50, while m = 2, 3, ... , 9

In [ ]:
for i in range(10,110,10):
    output_rf(i,10)

for i in range(2,10):
    output_rf(50,i)

## To sum up, the result of model is:

|number_of_tree | m | train_mse | test_mse | accuracy|
| --- | --- | --- | --- | --- |
|10 | 10 | 0.5335200200000003 | 0.7871440700000001 | 0.3467342858099355|
|20 | 10 | 0.4364876541666668 | 0.7240717475000001 | 0.5999057144983408|
|30 | 10 | 0.4098352755555559 | 0.7156564111111117 | 0.6334500431555337|
|40 | 10 | 0.407899104375 | 0.7187785887499998 | 0.6351655397341303|
|50 | 10 | 0.39257554666666655 | 0.6859994412 | 0.6506544917751644|
|60 | 10 | 0.3858063809259258 | 0.6635371325 | 0.6541760861143977|
|70 | 10 | 0.383716292244898 | 0.6566139267346943 | 0.6561511569752639|
|80 | 10 | 0.3812138177083334 | 0.6525010493750003 | 0.6556321867315669|
|90 | 10 | 0.3779110195061731 | 0.6414826459259259 | 0.6551598837463128|
|100 | 10 | 0.37467921310000035 | 0.6449070439000002 | 0.6557394687114719|
|50 | 2 | 0.5064935771999997 | 0.8488472079999996 | 0.526867202635738|
|50 | 3 | 0.43809759706666673 | 0.8125209203999998 | 0.5871346553018497|
|50 | 4 | 0.4157524953333331 | 0.6809749019999998 | 0.6153834534513313|
|50 | 5 | 0.3765126521333334 | 0.6351890164000001 | 0.657199068199613|
|50 | 6 | 0.3855411556000001 | 0.6474594047999999 | 0.6502836382766455|
|50 | 7 | 0.3644490563999996 | 0.6212765356000001 | 0.6638710749153269|
|50 | 8 | 0.39347622453333364 | 0.7214084211999999 | 0.6408934836836202|
|50 | 9 | 0.4100130346666663 | 0.6858754636000003 | 0.6328933984156652|

# (e) $Bias^2$ and Variance curve

## Steps

* We train 10 models with different value of features. The number of features are 10, 20, ..., 100

* Then we compute the error rate based on the original test dataset.

## Formula

### $Bias^2$

$$ Bias^2 = E_{(x,y)} [(\bar{h} (x) - t(x) )^2] $$

### Variance

$$ Variance = E_{(x,y)} [(h_D (x) - \bar{h} (x) )^2] $$

## Coding

### We first compute bias square and variance

In [ ]:
bias_sqr_list = list()
var_list = list()

X_train = train_x.values
X_test = test_x.values
y_train = train_y.values
y_test = test_y.values

for i in range(10,110,10):
    rf = RandomForestRegressor(max_features = "sqrt", random_state = 123, n_estimators = i)
    rf.fit(train_x, train_y)
    avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(rf, X_train, y_train, X_test, y_test, loss='mse',random_seed=123)
    bias_sqr_list.append(avg_bias**2)
    var_list.append(avg_var)

### Then we plot the $Bias^2$ graph

In [ ]:
fig, ax = plt.subplots()
ax.plot(range(10,110,10), bias_sqr_list, linewidth=2.0)
ax.scatter(range(10,110,10),bias_sqr_list)
ax.set(xlim=(10, 100), xticks=np.arange(10, 100,10), 
       ylim=(0, 4), yticks=np.arange(1, 4))
ax.set_title('Bias Square with different number of trees')
ax.set_ylabel('Bias square')
ax.set_xlabel('Number of Trees')

### Then plot the graph for variance

In [ ]:
fig, ax = plt.subplots()
ax.plot(range(10,110,10), var_list, linewidth=2.0)
ax.scatter(range(10,110,10),var_list)
ax.set(xlim=(10, 100), xticks=np.arange(10, 100,10), 
       ylim=(0, 1), yticks=np.arange(0, 1))
ax.set_title('Variance with different number of trees')
ax.set_ylabel('Variance')
ax.set_xlabel('Number of Trees')

## Analysis of the two graphs:

### For Bias:

In the graph, we can observe that there is no clear relationship between variance and bias. Also, for bias square, It does not wave a lot with different number of trees.

### For Variance:

From the graph above, we can draw that the variance of forest descendsmonotinously with the increase of number_of_trees. That is, the increase of tree number can help reduce variance to somehow reduce the total error.